In [11]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras.layers import SimpleRNN, LSTM, Bidirectional

# Simple RNN

In [2]:
# data
raw_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
train_X = np.array([raw_X]) # since input tensor of keras RNN layer must be 3D (batch_size, timesteps, input_dim)

print(np.shape(raw_X))
print(np.shape(train_X))

(4, 5)
(1, 4, 5)


In [3]:
# RNN
rnn = SimpleRNN(units=3)
hidden_states = rnn(train_X)
print(hidden_states)

rnn = SimpleRNN(units=3, return_sequences=True) # time series data
hidden_states = rnn(train_X)
print(hidden_states)

rnn = SimpleRNN(units=3, return_sequences=True, return_state=True) # return hidden states and last hidden state
hidden_states, last_state = rnn(train_X)
print(hidden_states)
print(last_state)

tf.Tensor([[-0.90406674 -0.95695645  0.4871061 ]], shape=(1, 3), dtype=float32)
tf.Tensor(
[[[ 0.53989905 -0.7305396   0.9989259 ]
  [ 0.56508136 -0.42520976  0.93777025]
  [ 0.40157634  0.5699616   0.20279834]
  [-0.7226658  -0.7673637  -0.7919916 ]]], shape=(1, 4, 3), dtype=float32)
tf.Tensor(
[[[-0.93144274 -0.98647815  0.95165825]
  [-0.9861711  -0.99771756  0.90277547]
  [-0.911559   -0.9881302   0.53207195]
  [-0.1292545  -0.99389213  0.9906109 ]]], shape=(1, 4, 3), dtype=float32)
tf.Tensor([[-0.1292545  -0.99389213  0.9906109 ]], shape=(1, 3), dtype=float32)


# LSTM (Long Short Term Memory)

In [7]:
lstm = LSTM(units=3, return_sequences=False, return_state=True)
hidden_state, last_hidden_state, last_cell_state = lstm(train_X)
print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

lstm = LSTM(units=3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[-0.14770149 -0.3586996   0.17888106]], shape: (1, 3)
last hidden state : [[-0.14770149 -0.3586996   0.17888106]], shape: (1, 3)
last cell state : [[-0.5037084  -0.5890213   0.21560201]], shape: (1, 3)
hidden states : [[[-0.05164738 -0.0841753   0.34916106]
  [-0.11765892  0.00512141  0.65895736]
  [-0.03715635  0.0396741   0.71605664]
  [-0.20584036  0.28282502  0.5444041 ]]], shape: (1, 4, 3)
last hidden state : [[-0.20584036  0.28282502  0.5444041 ]], shape: (1, 3)
last cell state : [[-0.44378722  0.5295171   1.237071  ]], shape: (1, 3)


# Bidirectional LSTM
## recurrent kernel is weights for the previous output
## kernel is weights for the current input

In [10]:
# fix initializing values
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [14]:
"""
hidden_state and cell_state of forward and backward
hidden_state = [forward_h, backward_h]

when return_sequences=False : hidden_state is combined of latest forward_h and backward_h
when return_sequences=True : hidden_state is combined of latest forward_h and first backward_h
"""
bi_lstm = Bidirectional(LSTM(units=3, kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init, return_sequences=False, return_state=True))

hidden_state, forward_h, forward_c, backward_h, backward_c = bi_lstm(train_X) 
print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('forward hidden state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward hidden state : {}, shape: {}'.format(backward_h, backward_h.shape))
print('forward cell state : {}, shape: {}'.format(forward_c, forward_c.shape))
print('backward cell state : {}, shape: {}'.format(backward_c, backward_c.shape))

bi_lstm = Bidirectional(LSTM(units=3, kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init, return_sequences=True, return_state=True))
hidden_states, forward_h, forward_c, backward_h, backward_c = bi_lstm(train_X)
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward hidden state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward hidden state : {}, shape: {}'.format(backward_h, backward_h.shape))
print('forward cell state : {}, shape: {}'.format(forward_c, forward_c.shape))
print('backward cell state : {}, shape: {}'.format(backward_c, backward_c.shape))

hidden state : [[0.63031393 0.63031393 0.63031393 0.7038734  0.7038734  0.7038734 ]], shape: (1, 6)
forward hidden state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward hidden state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)
forward cell state : [[1.5707088 1.5707088 1.5707088]], shape: (1, 3)
backward cell state : [[1.6402447 1.6402447 1.6402447]], shape: (1, 3)
hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.5511133  0.5511133  0.5511133  0.58863586 0.58863586 0.58863586]
  [0.59115756 0.59115756 0.59115756 0.3951699  0.3951699  0.3951699 ]
  [0.63031393 0.63031393 0.63031393 0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward hidden state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward hidden state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)
forward cell state : [[1.5707088 1.5707088 1.5707088]], shape: (1, 3)
backward cell state : [[1.6402447 1.6402447 1.6402447]], shape: (1, 3)
